In [2]:
import pandas as pd
import numpy as np
from sklearn import metrics

from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier

import numbers
from sklearn import model_selection


In [3]:

data_for_use = pd.read_csv('./ethereum_data_for_use.csv', index_col=0)
data_btc = pd.read_csv('./bitcoin_data_for_use_v2.csv', index_col=0)
data_gold = pd.read_csv('./gold_data_for_use.csv', index_col=0)
data_nasdaq = pd.read_csv('./nasdaq_data_for_use.csv', index_col=0)

data_for_use_w_gold = data_for_use.merge(data_gold, left_index=True, right_index=True)
data_for_use_w_nasdaq = data_for_use.merge(data_nasdaq, left_index=True, right_index=True)

data_for_use_w_all = data_for_use_w_gold.merge(data_nasdaq, left_index=True, right_index=True)

data_for_use_basic = data_for_use.drop(labels=['dia',
                    'varV0','varV1','varV2','varV3','varV4','varV5','varV6','varV7','varV8','varV9',
                    'varV10','varV11','varV12','varV13','varV14','varV15','varV16','varV17','varV18','varV19',
                    'varV20','varV21','varV22','varV23','varV24','varV25','varV26','varV27','varV28','varV29'
                    ], axis=1)

data_for_use_bone_deep = data_for_use_basic.drop(labels=['varPm93',
                    'varPm123','varPm153','varPm183','varPm213','varPm243','varPm273','varPm303','varPm333',
                    'varPs30','varPs37','varPs44','varPs51','varPs58','varPs65','varPs72','varPs79','varPs86'
                    ], axis=1)


In [4]:
data_for_use_basic

,varP0,varP1,varP2,varP3,varP4,varP5,varP6,varP7,varP8,varP9,...,varPm93,varPm123,varPm153,varPm183,varPm213,varPm243,varPm273,varPm303,varPm333,subida
2017-01-01,0.025532,-0.023392,-0.014990,0.101601,0.048710,-0.014197,0.013799,-0.012482,0.013901,-0.055221,...,0.132997,0.066791,-0.097853,-0.117164,0.475416,-0.198569,0.116500,3.115174,1.649285,True
2017-01-02,0.025199,0.025532,-0.023392,-0.014990,0.101601,0.048710,-0.014197,0.013799,-0.012482,0.013901,...,0.098192,0.365030,-0.250245,-0.153574,0.470898,-0.156268,0.013959,3.268243,1.713224,True
2017-01-03,0.160737,0.025199,0.025532,-0.023392,-0.014990,0.101601,0.048710,-0.014197,0.013799,-0.012482,...,0.089714,0.176716,-0.102615,-0.165163,0.398357,-0.058631,-0.083002,3.483239,1.670204,True
2017-01-04,0.156947,0.160737,0.025199,0.025532,-0.023392,-0.014990,0.101601,0.048710,-0.014197,0.013799,...,0.144038,0.064830,0.040654,-0.240687,0.495190,-0.125289,0.118582,2.769099,1.690495,False
2017-01-05,-0.088627,0.156947,0.160737,0.025199,0.025532,-0.023392,-0.014990,0.101601,0.048710,-0.014197,...,0.136739,0.069059,0.037658,-0.244379,0.487016,-0.069435,0.022594,2.321890,2.004026,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-21,-0.065753,-0.111266,-0.029886,0.009851,-0.048374,-0.012167,0.054273,0.043725,-0.036545,0.042766,...,-0.075806,0.027855,0.398158,-0.118883,0.711469,0.766659,0.137593,0.361917,0.278182,True
2021-09-22,0.113382,-0.065753,-0.111266,-0.029886,0.009851,-0.048374,-0.012167,0.054273,0.043725,-0.036545,...,-0.177400,-0.044863,0.431826,-0.132750,0.565372,1.118351,-0.040660,0.475192,0.180620,True
2021-09-23,0.025230,0.113382,-0.065753,-0.111266,-0.029886,0.009851,-0.048374,-0.012167,0.054273,0.043725,...,-0.111221,-0.107467,0.483348,-0.105825,0.447609,1.012714,0.012766,0.486636,0.153497,False
2021-09-24,-0.070941,0.025230,0.113382,-0.065753,-0.111266,-0.029886,0.009851,-0.048374,-0.012167,0.054273,...,-0.247336,0.195316,0.386287,0.016020,0.128337,1.221561,0.097644,0.448854,0.109653,False


In [5]:
#aqui se marca lo que se va a usar de verdad para el resto del programa
data_for_use = data_for_use_basic

In [6]:
# Partimos los datos en train y test por fechas, sin que sea aleatorio, que es como funcionaría
# en la realidad el modelo
data_no_test, data_test = train_test_split(data_for_use, shuffle=False, test_size=0.2)

# Partimos los datos de train nuevamente en validation y train
data_train, data_valid = train_test_split(data_no_test, shuffle=False, test_size=0.2)

In [7]:
# Separamos la variable que queremos predecir
X_train=data_train.drop(labels=['subida'], axis=1)
y_train=data_train['subida']

X_valid=data_valid.drop(labels=['subida'], axis=1)
y_valid=data_valid['subida']

X_test=data_test.drop(labels=['subida'], axis=1)
y_test=data_test['subida']

In [8]:
# Miramos cuántos días sube el eth sobre el total en cada uno de los segmentos
# esto nos servirá para hacernos una idea de cómo de buenos son los resultados de los modelos
y_train.sum()/y_train.size

0.49909584086799275

In [9]:
y_valid.sum()/y_valid.size

0.5703971119133574

In [10]:
y_test.sum()/y_test.size

0.5664739884393064

In [11]:
#Tenemos que hacer one-hot encoding sobre la columna "dia"
#Sobre el resto aplicamos el standardscaler

number_columns = data_for_use.select_dtypes('number').columns

t=[
    #('dia', 
    #OneHotEncoder(handle_unknown='ignore'),
    #['dia']),
    ('scaler', StandardScaler(),number_columns)
    ]

ct = ColumnTransformer(transformers=t, remainder='passthrough')

#data_for_use_t=ct.fit_transform(data_for_use)

#X_train=ct.fit_transform(X_train)
#X_valid=ct.transform(X_valid)
#X_test=ct.transform(X_test)

In [12]:
def walk_forward_validation (model, X_train_wfv, y_train_wfv, X_test_wfv, y_test_wfv):
    y_pred_wfv = list()
    for i in range(len(y_test_wfv)):
        X_train_wfv_ct=ct.fit_transform(X_train_wfv)
        model.fit(X_train_wfv_ct, y_train_wfv)
        X_test_wfv_ct=ct.transform(X_test_wfv)
        y_pred_next = model.predict(X_test_wfv_ct[i:i+1])
        y_pred_wfv.append(y_pred_next[0])
        X_train_wfv=X_train_wfv.append(X_test_wfv[i:i+1])
        y_train_wfv=y_train_wfv.append(pd.Series(y_test_wfv[i]))
    
    return metrics.confusion_matrix(y_test_wfv, y_pred_wfv), metrics.f1_score(y_test_wfv,y_pred_wfv), metrics.accuracy_score(y_test_wfv,y_pred_wfv)
        
        

In [13]:
lr = LogisticRegression(penalty='l2', solver='lbfgs')

X_train_c=X_train.copy()
y_train_c=y_train.copy()
X_valid_c=X_valid.copy()
y_valid_c=y_valid.copy()


conf_mat, f1sc, accsc = walk_forward_validation(lr, X_train_c, y_train_c, X_valid_c, y_valid_c)

In [14]:
print(conf_mat, f1sc, accsc)

[[57 62]
 [76 82]] 0.5430463576158941 0.5018050541516246


In [15]:
for vecinos in range(1,31):
    kn = KNeighborsClassifier(n_neighbors=vecinos)
    
    X_train_c=X_train.copy()
    y_train_c=y_train.copy()
    X_valid_c=X_valid.copy()
    y_valid_c=y_valid.copy()


    conf_mat, f1sc, accsc = walk_forward_validation(kn, X_train_c, y_train_c, X_valid_c, y_valid_c)
    print ("Numero de vecinos aplicado: " + str(vecinos))
    print(conf_mat, f1sc, accsc)

Numero de vecinos aplicado: 1
[[60 59]
 [81 77]] 0.5238095238095238 0.49458483754512633
Numero de vecinos aplicado: 2
[[ 85  34]
 [124  34]] 0.3008849557522124 0.4296028880866426
Numero de vecinos aplicado: 3
[[58 61]
 [76 82]] 0.5448504983388704 0.5054151624548736
Numero de vecinos aplicado: 4
[[ 84  35]
 [106  52]] 0.42448979591836733 0.49097472924187724
Numero de vecinos aplicado: 5
[[61 58]
 [73 85]] 0.5647840531561461 0.5270758122743683
Numero de vecinos aplicado: 6
[[82 37]
 [93 65]] 0.5 0.5306859205776173
Numero de vecinos aplicado: 7
[[65 54]
 [67 91]] 0.6006600660066006 0.5631768953068592
Numero de vecinos aplicado: 8
[[82 37]
 [94 64]] 0.49420849420849416 0.5270758122743683
Numero de vecinos aplicado: 9
[[65 54]
 [68 90]] 0.5960264900662252 0.5595667870036101
Numero de vecinos aplicado: 10
[[77 42]
 [85 73]] 0.5347985347985347 0.5415162454873647
Numero de vecinos aplicado: 11
[[60 59]
 [74 84]] 0.5581395348837209 0.51985559566787
Numero de vecinos aplicado: 12
[[77 42]
 [89 6

In [16]:
for grado in range(1,4):
    sv = SVC(probability=True, kernel='poly', degree=grado)
    
    X_train_c=X_train.copy()
    y_train_c=y_train.copy()
    X_valid_c=X_valid.copy()
    y_valid_c=y_valid.copy()


    conf_mat, f1sc, accsc = walk_forward_validation(sv, X_train_c, y_train_c, X_valid_c, y_valid_c)
    print ("Grado aplicado: " + str(grado))
    print(conf_mat, f1sc, accsc)

Grado aplicado: 1
[[63 56]
 [77 81]] 0.5491525423728814 0.51985559566787
Grado aplicado: 2
[[44 75]
 [79 79]] 0.5064102564102564 0.44404332129963897
Grado aplicado: 3
[[ 41  78]
 [ 48 110]] 0.6358381502890174 0.5451263537906137


In [17]:
for profund in range(1,31):
    dt = DecisionTreeClassifier(max_depth=profund)
    
    X_train_c=X_train.copy()
    y_train_c=y_train.copy()
    X_valid_c=X_valid.copy()
    y_valid_c=y_valid.copy()


    conf_mat, f1sc, accsc = walk_forward_validation(dt, X_train_c, y_train_c, X_valid_c, y_valid_c)
    print ("Maxima profundidad aplicada: " + str(profund))
    print(conf_mat, f1sc, accsc)

Maxima profundidad aplicada: 1
[[ 98  21]
 [112  46]] 0.4088888888888889 0.51985559566787
Maxima profundidad aplicada: 2
[[ 97  22]
 [110  48]] 0.42105263157894735 0.5234657039711191
Maxima profundidad aplicada: 3
[[111   8]
 [136  22]] 0.23404255319148937 0.48014440433212996
Maxima profundidad aplicada: 4
[[100  19]
 [118  40]] 0.3686635944700461 0.5054151624548736
Maxima profundidad aplicada: 5
[[75 44]
 [92 66]] 0.4925373134328358 0.5090252707581228
Maxima profundidad aplicada: 6
[[ 81  38]
 [102  56]] 0.4444444444444444 0.49458483754512633
Maxima profundidad aplicada: 7
[[67 52]
 [94 64]] 0.4671532846715329 0.4729241877256318
Maxima profundidad aplicada: 8
[[60 59]
 [83 75]] 0.5136986301369862 0.48736462093862815
Maxima profundidad aplicada: 9
[[65 54]
 [92 66]] 0.47482014388489213 0.4729241877256318
Maxima profundidad aplicada: 10
[[62 57]
 [88 70]] 0.49122807017543857 0.47653429602888087
Maxima profundidad aplicada: 11
[[67 52]
 [86 72]] 0.5106382978723404 0.5018050541516246
Maxi

In [18]:
for estimadores in range(1,21):
    adab = AdaBoostClassifier(n_estimators=estimadores)
    
    X_train_c=X_train.copy()
    y_train_c=y_train.copy()
    X_valid_c=X_valid.copy()
    y_valid_c=y_valid.copy()


    conf_mat, f1sc, accsc = walk_forward_validation(adab, X_train_c, y_train_c, X_valid_c, y_valid_c)
    print ("Estimadores aplicados: " + str(estimadores))
    print(conf_mat, f1sc, accsc)

Estimadores aplicados: 1
[[ 98  21]
 [112  46]] 0.4088888888888889 0.51985559566787
Estimadores aplicados: 2
[[ 96  23]
 [110  48]] 0.4192139737991266 0.51985559566787
Estimadores aplicados: 3
[[ 91  28]
 [109  49]] 0.41702127659574467 0.5054151624548736
Estimadores aplicados: 4
[[77 42]
 [86 72]] 0.5294117647058824 0.5379061371841155
Estimadores aplicados: 5
[[73 46]
 [92 66]] 0.4888888888888889 0.5018050541516246
Estimadores aplicados: 6
[[71 48]
 [74 84]] 0.5793103448275861 0.5595667870036101
Estimadores aplicados: 7
[[71 48]
 [87 71]] 0.5126353790613719 0.5126353790613718
Estimadores aplicados: 8
[[66 53]
 [82 76]] 0.529616724738676 0.5126353790613718
Estimadores aplicados: 9
[[70 49]
 [81 77]] 0.5422535211267605 0.5306859205776173
Estimadores aplicados: 10
[[69 50]
 [70 88]] 0.5945945945945945 0.5667870036101083
Estimadores aplicados: 11
[[68 51]
 [76 82]] 0.563573883161512 0.5415162454873647
Estimadores aplicados: 12
[[64 55]
 [79 79]] 0.541095890410959 0.516245487364621
Estimado

In [19]:
dt = DecisionTreeClassifier(max_depth=2)

X_train_c=X_train.append(X_valid)
y_train_c=y_train.append(y_valid)

X_train_c=ct.fit_transform(X_train_c)
dt.fit(X_train_c,y_train_c)
y_predict=dt.predict(X_train_c)

(metrics.confusion_matrix(y_train_c, y_predict), metrics.f1_score(y_train_c,y_predict), metrics.accuracy_score(y_train_c,y_predict))

(array([[514, 159],
        [451, 259]]),
 0.45921985815602834,
 0.5589298626174982)

In [20]:
sv = SVC(probability=True, kernel='poly', degree=3)

X_train_c=X_train.append(X_valid)
y_train_c=y_train.append(y_valid)

X_train_c=ct.fit_transform(X_train_c)
sv.fit(X_train_c,y_train_c)
y_predict=sv.predict(X_train_c)

(metrics.confusion_matrix(y_train_c, y_predict), metrics.f1_score(y_train_c,y_predict), metrics.accuracy_score(y_train_c,y_predict))

(array([[388, 285],
        [  0, 710]]),
 0.8328445747800587,
 0.7939262472885033)

In [21]:
adab = AdaBoostClassifier(n_estimators=6)

X_train_c=X_train.append(X_valid)
y_train_c=y_train.append(y_valid)

X_train_c=ct.fit_transform(X_train_c)
adab.fit(X_train_c,y_train_c)
y_predict=adab.predict(X_train_c)

(metrics.confusion_matrix(y_train_c, y_predict), metrics.f1_score(y_train_c,y_predict), metrics.accuracy_score(y_train_c,y_predict))

(array([[440, 233],
        [341, 369]]),
 0.5624999999999999,
 0.5849602313810557)

In [22]:
adab = AdaBoostClassifier(n_estimators=10)

X_train_c=X_train.append(X_valid)
y_train_c=y_train.append(y_valid)

X_train_c=ct.fit_transform(X_train_c)
adab.fit(X_train_c,y_train_c)
y_predict=adab.predict(X_train_c)

(metrics.confusion_matrix(y_train_c, y_predict), metrics.f1_score(y_train_c,y_predict), metrics.accuracy_score(y_train_c,y_predict))

(array([[350, 323],
        [217, 493]]),
 0.6461336828309304,
 0.6095444685466378)

In [23]:
adab = AdaBoostClassifier(n_estimators=19)

X_train_c=X_train.append(X_valid)
y_train_c=y_train.append(y_valid)

X_train_c=ct.fit_transform(X_train_c)
adab.fit(X_train_c,y_train_c)
y_predict=adab.predict(X_train_c)

(metrics.confusion_matrix(y_train_c, y_predict), metrics.f1_score(y_train_c,y_predict), metrics.accuracy_score(y_train_c,y_predict))

(array([[417, 256],
        [233, 477]]),
 0.661122661122661,
 0.6464208242950108)

In [24]:
kn = KNeighborsClassifier(n_neighbors=7)

X_train_c=X_train.append(X_valid)
y_train_c=y_train.append(y_valid)

X_train_c=ct.fit_transform(X_train_c)
kn.fit(X_train_c,y_train_c)
y_predict=kn.predict(X_train_c)

(metrics.confusion_matrix(y_train_c, y_predict), metrics.f1_score(y_train_c,y_predict), metrics.accuracy_score(y_train_c,y_predict))

(array([[430, 243],
        [262, 448]]),
 0.6395431834403997,
 0.6348517715112075)

In [25]:
kn = KNeighborsClassifier(n_neighbors=9)

X_train_c=X_train.append(X_valid)
y_train_c=y_train.append(y_valid)

X_train_c=ct.fit_transform(X_train_c)
kn.fit(X_train_c,y_train_c)
y_predict=kn.predict(X_train_c)

(metrics.confusion_matrix(y_train_c, y_predict), metrics.f1_score(y_train_c,y_predict), metrics.accuracy_score(y_train_c,y_predict))

(array([[415, 258],
        [238, 472]]),
 0.6555555555555556,
 0.6413593637020969)

In [27]:
#Este es el código para ver cómo funciona el modelo de verdad (este es el resultado del basic)

adab = AdaBoostClassifier(n_estimators=10)

X_train_c=X_train.append(X_valid)
y_train_c=y_train.append(y_valid)
X_test_c=X_test.copy()
y_test_c=y_test.copy()

conf_mat, f1sc, accsc = walk_forward_validation(adab, X_train_c, y_train_c, X_test_c, y_test_c)

print(conf_mat, f1sc, accsc)

[[ 68  82]
 [ 70 126]] 0.6237623762376238 0.5606936416184971
